In [ ]:
pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 파일은 4가지 버전이 있음.
- label 88개 : {1024, 256 버전}
- label 30개(good, bad) : {1024, 256 버전}

In [ ]:
# 증강 방법 정의
def vertical_flip(img, flag):
    if flag:
        return cv2.flip(img, 0)
    else:
        return img

def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img


def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img

def vertical_shift_up(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(0.0, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    img = fill(img, h, w)
    return img


def random_rotation(img, angle):
  angle_list = [int(angle), 15, 30, 45, 60, 90, 180]
  angle = random.choice(angle_list)
  h, w = img.shape[:2]
  M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
  img = cv2.warpAffine(img, M, (w, h))
  return img

def contrast(gray, min_val, max_val):
    #gray = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    alpha = int(random.uniform(min_val, max_val)) # Contrast control
    adjusted = cv2.convertScaleAbs(gray, alpha=alpha)
    return adjusted 

def brightness(gray, val):
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    brightness = int(random.uniform(-val, val))
    if brightness > 0:
        gray = gray + brightness
    else:
        gray = gray - brightness
    gray = np.clip(gray, 10, 255)
    return gray     

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,  # 이미지 크기 변경, RGB값을 0~1로 정규화
                                  horizontal_flip=True,          # 수평 대칭 이미지지 생성
                                  width_shift_range=0.1,    # 전체 크기의 10% 범위에서 좌우 이동
                                  height_shift_range=0.1,   # 전체 크기의 10% 범위에서 위, 아래로 이동
                                  rotation_range=22.5,              # 정해진 각도 만큼 이미지 회전
                                  shear_range=0.7,               # 좌표 하나 고정하고, 다른 몇개의 좌표를 이동시켜 변환
                                  vertical_flip=True,               # 수직 대칭 이미지 생성
                                  fill_mode='nearest')  



NameError: ignored

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os, glob
import random
from tqdm import tqdm
from time import time

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import PIL
from PIL import Image
import torch
import torchvision
from torch.utils.data import  TensorDataset, DataLoader

In [ ]:
# 2. 데이터 불균형 확인
#파일 경로 가져오기
PATH = '/content/drive/MyDrive/DL_Project/train_good_bad'

image_dirs = os.listdir(PATH)
image_paths = dict()
for image_dir in tqdm(image_dirs):
    try:
        files = glob.glob(PATH + image_dir+"/*.png") 
        image_paths[image_dir] = [file for file in files]
    except:
        FileNotFoundError

100%|██████████| 30/30 [00:00<00:00, 4402.54it/s]


In [ ]:
image_dir

'wood-good'

In [ ]:
data = dict()
data['label'] = []
data['filename'] = []

for label,paths in tqdm(image_paths.items()):
    for path in paths:
        img = Image.open(path)
        data['filename'].append(path)
        # 파일 경로에서 클래스 레이블 추출하여 추가
        data['label'].append(label)

100%|██████████| 30/30 [00:00<00:00, 10846.40it/s]


In [ ]:
data['label']

[]

In [ ]:
#파일 정보 확인 및 이미지 확인, data imbalance 확인 
from collections import Counter
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
# print('number of samples: ', len(data['image_arr']))
print('dict keys: ', list(data.keys()))
# print('image shape: ', data['image_arr'][0].shape)
count_label = Counter(data['label'])
print('Original dataset shape')
print(sorted(count_label.items()))

dict keys:  ['label', 'filename']
Original dataset shape
[]


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import time
from tqdm import tqdm
from collections import Counter

PATH = '/content/drive/MyDrive/DL_Project/train_class_256/pill-crack'
image_dirs = os.listdir(PATH)

label_count = Counter(data['label'])
label_max = max(label_count.values())

# 이미지 증강 함수 정의
def augment_image(img):
    img = contrast(img, 1, 1.5) # 대비 조절
    img = horizontal_flip(img, 1) # 좌우 반전
    img = vertical_flip(img, 1)
    img = random_rotation(img, 45) # 임의의 각도로 회전
    img = horizontal_shift(img, 0.1) # 좌우 이동
    return img

for image_dir in tqdm(image_dirs):
    image_list = os.listdir(PATH+image_dir+"/")
    len_image_list = len(image_list)

    while(len_image_list<label_max):   # 100개까지 이미지 생성
        
        for image in image_list:
            img = cv2.imread(PATH+image_dir+"/"+image)
            img = augment_image(img) # 이미지 증강
            file_name = PATH+image_dir+"/arumentation_"+str(time.time())+ '.png'
            cv2.imwrite(file_name, img)
            len_image_list += 1
            if len_image_list >= label_max:
                break;

    len_image_list = 0

ValueError: ignored

In [ ]:
label_count = Counter(data['label'])
label_count.values()

dict_values([])

### 위의 파일이 뭔가 안 되서 각각 파일 별로 증강시키는 코드를 만듬 

참고로 88개는 각각 100개, 30개는 200개 정도로 증강

In [14]:
from PIL import Image
import cv2
import numpy as np
import random

def image_augmentation(image):
    # 이미지 크기 변경
    resized_image = cv2.resize(image, (random.randint(100, 500), random.randint(100, 500)))

    # 이미지 회전
    (h, w) = resized_image.shape[:2]
    center = (w // 2, h // 2)
    angle = random.randint(0, 360)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(resized_image, M, (w, h))

    # 이미지 대칭
    if random.random() > 0.5:
        flipped_image = cv2.flip(rotated_image, 1)
    else:
        flipped_image = rotated_image

    return flipped_image

In [16]:
import os

# 이미지 파일 경로 설정
image_dir = '/content/drive/MyDrive/DL_Project/train_256_good_bad/cable-bad'

# 이미지 파일 불러오기
image_paths = [os.path.join(image_dir, file) for file in os.listdir(image_dir)]

# 이미지 증강하기
for i in range(200):
    # 랜덤한 이미지 선택
    image_path = random.choice(image_paths)

    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 이미지 증강
    augmented_image = image_augmentation(image)

    # 이미지 저장
    new_image_path = f'/content/drive/MyDrive/DL_Project/train_256_argu/bottle_bad/{i}.png'
    cv2.imwrite(new_image_path, augmented_image)

In [ ]:
image_paths

['/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/12787.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/11273.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/13112.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/10759.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/10053.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/11763.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/11307.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/13462.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/12276.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/10837.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/12757.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-bad/10244.png',
 '/content/drive/MyDrive/DL_Project/train_256_good_bad/bottle-ba

추가로 이 이미지들로 전이학습을 해줘야 하는데, 어떤 방식으로 해야 되는가?